**author:** Alex Dawson-Elli  
**assignment:** Homework 1  
**due:** 2/2/2018



**Problem 1:**  solve the following linear program using Clp Ecos and SCS solvers. Compare the answers found by each solver

objective fxn:  
$5x_1 - x_2 + 11x_3$  

constraints:  
  $2x1 ≥ x2 + x3$  
  $ ≤ xj ≤ 3, j ∈ {1, 2, 3}$



In [1]:
#define the problem, independent of solver used:
using JuMP, Clp 


"""solve problem 1 with the supplied solver type"""
function solve(model)
    
end

LoadError: syntax: invalid "using" statement: expected identifier

In [8]:
using JuMP, Clp

m = Model(solver = ClpSolver())

@variable(m, 0 <= f <= 1000)           # football trophies
@variable(m, 0 <= s <= 1500)           # soccer trophies
@constraint(m, 4f + 2s <= 4800)        # total board feet of wood
@constraint(m, f + s <= 1750)          # total number of plaques
@objective(m, Max, 12f + 9s)           # maximize profit

status = solve(m)

println(status)
println("Build ", getvalue(f), " football trophies.")
println("Build ", getvalue(s), " soccer trophies.")
println("Total profit will be \$", getobjectivevalue(m))

INFO: Precompiling module Clp.
ERROR: LoadError: Cbc not properly installed. Please run Pkg.build("Cbc")
 in error(::String) at .\error.jl:21
 in include_from_node1(::String) at .\loading.jl:488
 in macro expansion; at .\none:2 [inlined]
 in anonymous at .\<missing>:?
 in eval(::Module, ::Any) at .\boot.jl:234
 in process_options(::Base.JLOptions) at .\client.jl:242
 in _start() at .\client.jl:321
while loading C:\Users\Alex\AppData\Local\JuliaPro-0.5.1.1\pkgs-0.5.1.1\v0.5\Cbc\src\Cbc.jl, in expression starting on line 8
ERROR: LoadError: LoadError: Failed to precompile Cbc to C:\Users\Alex\AppData\Local\JuliaPro-0.5.1.1\pkgs-0.5.1.1\lib\v0.5\Cbc.ji.
 in compilecache(::String) at .\loading.jl:593
 in require(::Symbol) at .\loading.jl:393
 in include_from_node1(::String) at .\loading.jl:488 (repeats 2 times)
 in macro expansion; at .\none:2 [inlined]
 in anonymous at .\<missing>:?
 in eval(::Module, ::Any) at .\boot.jl:234
 in process_options(::Base.JLOptions) at .\client.jl:242
 in _st

LoadError: Failed to precompile Clp to C:\Users\Alex\AppData\Local\JuliaPro-0.5.1.1\pkgs-0.5.1.1\lib\v0.5\Clp.ji.